In [1]:
import tensorflow as tf
from keras import backend as K
# from normal_lstm import base_model, my_generator
# model_folder ="/home/kei666/tomo/lstm-cnn/data/normal_lstm/0/"
from max_pooling import base_model, my_generator
model_folder ="/home/kei666/tomo/lstm-cnn/data/max_pooling/0/"

import matplotlib.pyplot as plt

import numpy as np
import os
import json

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
#カメラの数
camera=5

#データの場所
data_folder ="/home/kei666/tomo/lstm-cnn/data/"

#画像関連 
channels=3
img_width=50
img_height=50

timesteps=10

In [3]:
train_path = data_folder + "test0/train/"
label_path = data_folder + "test0/label/"

list1 = os.listdir(train_path)
train_paths = []
label_paths = []
for i in range(len(list1)):
    train_paths.append(train_path+str(i)+".npy")
    label_paths.append(label_path+str(i)+".npy")
print(len(train_paths))

10815


In [4]:
# モデル定義
pred = base_model.Prediction(timesteps, camera, img_width, img_height)
model = pred.create_model()
print(model.summary())












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 21, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 21, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 21, 50, 50, 3 0                                            
_____________________________________________________________________________________

In [ ]:
val_count = int(len(train_paths) * 0.2)
train_gen = my_generator.MyGenerator(
                             train_paths[val_count:], 
                             label_paths[val_count:], 
                             batch_size=5)
val_gen = my_generator.MyGenerator(
                             train_paths[:val_count], 
                             label_paths[:val_count], 
                             batch_size=5)
history = model.fit_generator(
    train_gen, 
    steps_per_epoch=train_gen.num_batches_per_epoch, 
    validation_data=val_gen,
    validation_steps=val_gen.num_batches_per_epoch,
    epochs=100,
    class_weight={0:1,1:4},
    shuffle=True)



Epoch 1/100
1731/1731 [==============================] - 1148s 663ms/step - loss: 3.3696 - shared_Ratelayers_loss: 0.6739 - shared_Ratelayers_acc: 0.6193 - shared_Ratelayers_acc_1: 0.6191 - shared_Ratelayers_acc_2: 0.6195 - shared_Ratelayers_acc_3: 0.6194 - shared_Ratelayers_acc_4: 0.6192 - val_loss: 3.8819 - val_shared_Ratelayers_loss: 0.7764 - val_shared_Ratelayers_acc: 0.4646 - val_shared_Ratelayers_acc_1: 0.4646 - val_shared_Ratelayers_acc_2: 0.4646 - val_shared_Ratelayers_acc_3: 0.4646 - val_shared_Ratelayers_acc_4: 0.4646
Epoch 2/100
 110/1731 [>.............................] - ETA: 14:27 - loss: 3.3049 - shared_Ratelayers_loss: 0.6610 - shared_Ratelayers_acc: 0.6218 - shared_Ratelayers_acc_1: 0.6218 - shared_Ratelayers_acc_2: 0.6218 - shared_Ratelayers_acc_3: 0.6218 - shared_Ratelayers_acc_4: 0.6218

In [ ]:
model.save(model_folder+'sample.h5')
with open(model_folder+'history.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()